<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [43]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [44]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "2ZoLYydQfIz5VW5b.D5B2X2LLCmJsysb4n78mTmGgEAWzlLfrgRKcCLJXlE4iJZ9Mqhm1GhsG60PJQxls"

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1158296
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "sweden", "city": "sweden", "street": "sollentuna-sollentunav\\u00e4gen 192", "aqicn_url": "https://api.waqi.info/feed/@13984", "latitude": 17.950061509561397, "longitude": 59.433619224293736}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [45]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [46]:
import requests
import pandas as pd
# Read the air quality feature group into a pandas dataframe
air_quality_df = air_quality_fg.read()
# Get the last date in the air quality feature group
last_date = air_quality_df['date'].max()
print(last_date)

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
# Convert the date column to include timezone information
aq_today_df['date'] = pd.to_datetime(aq_today_df['date'], utc=True)
print(aq_today_df)

# Merge the lag columns into the aq_today_df
aq_today_df = aq_today_df.merge(air_quality_df[['date', 'pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3']], on='date', how='left')

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.97s) 
2024-11-13 00:00:00+00:00
   pm25 country    city                          street  \
0  17.0  sweden  sweden  sollentuna-sollentunavägen 192   

                       date                                url  
0 2024-11-13 00:00:00+00:00  https://api.waqi.info/feed/@13984  


,pm25,country,city,street,date,url,pm25_lag_1,pm25_lag_2,pm25_lag_3
0,17.0,sweden,sweden,sollentuna-sollentunavägen 192,2024-11-13 00:00:00+00:00,https://api.waqi.info/feed/@13984,13.0,16.0,10.0


In [47]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   pm25        1 non-null      float32            
 1   country     1 non-null      object             
 2   city        1 non-null      object             
 3   street      1 non-null      object             
 4   date        1 non-null      datetime64[ns, UTC]
 5   url         1 non-null      object             
 6   pm25_lag_1  1 non-null      float32            
 7   pm25_lag_2  1 non-null      float32            
 8   pm25_lag_3  1 non-null      float32            
dtypes: datetime64[ns, UTC](1), float32(4), object(4)
memory usage: 184.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [48]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 18.0°N 59.5°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-13,27.200001,0.0,5.506941,11.309896,sweden
1,2024-11-14,27.049999,0.0,16.199999,36.869991,sweden
2,2024-11-15,28.350000,0.0,19.881649,42.064255,sweden
3,2024-11-16,27.500000,0.0,21.385939,36.098385,sweden
4,2024-11-17,27.400000,0.0,15.277749,34.439060,sweden
5,2024-11-18,28.150000,0.0,26.242987,63.083534,sweden
6,2024-11-19,29.000000,0.0,22.383780,62.198643,sweden
7,2024-11-20,27.950001,0.1,24.696104,59.322803,sweden
8,2024-11-21,27.500000,0.0,19.228851,51.842735,sweden
9,2024-11-22,26.850000,0.0,17.057314,44.145004,sweden


In [49]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [50]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-13 15:24:27,758 INFO: 	4 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1158296/fs/1148999/fg/1346950


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1158296/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2c20eee90>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_lag_3",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 670773
         }
       },
       "result": {
         "observed_value": 10.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T02:24:27.000757Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
     

In [51]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-13 15:24:37,666 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1158296/fs/1148999/fg/1346951


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1158296/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x293e1e4d0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 670775
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T02:24:37.000666Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
